# Информация импорта с Kaggle

In [ ]:
import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

# Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input

# Загрузка данных

In [ ]:
# Загрузка обучающей выборки
train_data = pd.read_csv('dataset/fashion-mnist_train.csv/fashion-mnist_train.csv')

# Загрузка тестовой выборки
test_data = pd.read_csv('dataset/fashion-mnist_test.csv/fashion-mnist_test.csv')

# Проверка на наличие NaN значений

In [ ]:
# Проверка на наличие NaN значений в обучающей выборке
print("Общее количество пропущенных значений в обучающей выборке:",
      train_data.isnull().sum().sum())

# Проверка на наличие NaN значений в тестовой выборке
print("Общее количество пропущенных значений в тестовой выборке:",
      test_data.isnull().sum().sum())

Общее количество пропущенных значений в обучающей выборке: 0
Общее количество пропущенных значений в тестовой выборке: 0


# Подготовка данных

In [ ]:
# Извлечение меток классов и изображений из обучающей выборки
train_labels = train_data['label']
train_images = train_data.drop(['label'], axis=1)

# Извлечение меток классов и изображений из тестовой выборки
test_labels = test_data['label']
test_images = test_data.drop(['label'], axis=1)

# Предварительная обработка изображений

In [ ]:
# Преобразование изображений в двумерные массивы
train_images = train_images.values.reshape(-1, 28, 28)
test_images = test_images.values.reshape(-1, 28, 28)

# Нормализация данных

In [ ]:
# Нормализация пикселей изображений
train_images = train_images / 255.0
test_images = test_images / 255.0

# Создание модели

In [ ]:
model = Sequential([
    # Использование объекта Input для определения формы входных данных
    Input(shape=(28, 28, 1)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

# Компилирование модели
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Вывод сводки модели
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 5408)              0         
                                                                 
 dense (Dense)               (None, 64)                346176    
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 347146 (1.32 MB)
Trainable params: 347146 (1.32 MB)
Non-trainable params: 0 (0.00 Byte)
____________________

# Обучение модели

In [ ]:
model.fit(train_images, train_labels, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 9s 3ms/step - loss: 0.4141 - accuracy: 0.8537
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2830 - accuracy: 0.8983
Epoch 3/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2398 - accuracy: 0.9135
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2104 - accuracy: 0.9229
Epoch 5/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1857 - accuracy: 0.9324
Epoch 6/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1649 - accuracy: 0.9396
Epoch 7/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1459 - accuracy: 0.9461
Epoch 8/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1297 - accuracy: 0.9521
Epoch 9/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1131 - accuracy: 0.9585
Epoch 10/10
1875/1875 [==============================] - 6s 3ms/step - loss: 0.102

# Предсказание и сохранение файла submission.csv для отправки

In [ ]:
# Предсказание вероятностей классов для тестовой выборки
predictions_probabilities = model.predict(test_images)

# Преобразование вероятностей в классы, выбирая класс с наибольшей вероятностью
predictions = np.argmax(predictions_probabilities, axis=1)

# Создание DataFrame для отправки
submission = pd.DataFrame({
    'Id': range(len(predictions)),  # Создаем столбец 'id' с числами от 0 до len(predictions)-1
    'Category': predictions
})

# Сохранение файла для отправки
submission.to_csv('submission.csv', index=False)

313/313 [==============================] - 1s 2ms/step
